In [1]:
import pandas as pd
import numpy as np
#import geopandas as gpd
import urllib#pour récupérer les données
import bs4#pour rendre lisibles les données
import re

Récupération des députés sur le site de l'Assemblée Nationale

In [2]:
def Scrap (url):
    req = urllib.request.Request(url)
    html = urllib.request.urlopen(req).read()
    page = bs4.BeautifulSoup(html, "lxml")
    return page

In [3]:
def ToDf (table) :
    Listgrossiere=pd.read_html(str(table))#on transforme le tableau en liste de dataframes
    listeDf=Listgrossiere[0]#on récupère le dataframe
    return listeDf

In [4]:
page=Scrap('http://www2.assemblee-nationale.fr/sycomore/resultats/(offset)/'+str(0)+'/(query)/IiBTRUxFQ1QgbV9jb2RlX2RlcHV0ZSwgaWRfYWN0ZXVyX3RyaWJ1biwgbGVnX21heF90cmlidW4sIG5vbSwgbm9tX2FmZmljaGUsIHByZW5vbSwgZGF0ZV9uYWlzLCBkYXRlX2RlY2VzIEZST00gZGVwdXRlIFdIRVJFIDE9MSAgQU5EIG1fY29kZV9kZXB1dGUgSU4gKFNFTEVDVCBtX2NvZGVfZGVwdXRlIEZST00gbWFuZGF0LCBkZXBhcnRlbWVudCAgV0hFUkUgbWFuZGF0Lm1fbnVtX2RlcCA9IGRlcGFydGVtZW50Lm1fbnVtX2RlcCBBTkQgbWFuZGF0Lm1fdHlwZV9tYW5kYXQgPSAxICkgT1JERVIgQlkgZGF0ZV9uYWlzIERFU0Mi')
table = page.find('table')
liste=ToDf(table)
for i in range(32):
    page=Scrap('http://www2.assemblee-nationale.fr/sycomore/resultats/(offset)/'+str((i+1)*500)+'/(query)/IiBTRUxFQ1QgbV9jb2RlX2RlcHV0ZSwgaWRfYWN0ZXVyX3RyaWJ1biwgbGVnX21heF90cmlidW4sIG5vbSwgbm9tX2FmZmljaGUsIHByZW5vbSwgZGF0ZV9uYWlzLCBkYXRlX2RlY2VzIEZST00gZGVwdXRlIFdIRVJFIDE9MSAgQU5EIG1fY29kZV9kZXB1dGUgSU4gKFNFTEVDVCBtX2NvZGVfZGVwdXRlIEZST00gbWFuZGF0LCBkZXBhcnRlbWVudCAgV0hFUkUgbWFuZGF0Lm1fbnVtX2RlcCA9IGRlcGFydGVtZW50Lm1fbnVtX2RlcCBBTkQgbWFuZGF0Lm1fdHlwZV9tYW5kYXQgPSAxICkgT1JERVIgQlkgZGF0ZV9uYWlzIERFU0Mi')
    table = page.find('table')#on récupère le seul tableau de la page qui liste des députés
    listeDf=ToDf(table)
    liste=pd.concat([liste,listeDf])#on récupère le dataframe et on l'ajoute à la liste
    
liste=liste.reset_index()#on recrée un index afin de ne pas s'arrêter à 500 puis recommencer comme dans les pages web
listeDepute=liste[['Nom','Né le']]#on récupère les variables qui nous intéressent  

In [5]:
listeDepute.head()

,Nom,Né le
0,Ludovic Pajot,18 novembre 1993
1,Typhanie Degois,6 janvier 1993
2,Lénaïck Adam,19 février 1992
3,Pierre Henriet,26 novembre 1991
4,Robin Reda,10 mai 1991


On s'intéresse ensuite à la récupération des mandats de chaque député (partie en cours).

In [8]:
def scrap_un_mandat(mandat):
    #On récupère et lave les titres des colonnes
    columns=mandat.findAll('dt')
    columns=re.sub('<[^<]+?>', '', str(columns))  
    
    #On récupère et lave les titres des lignes
    lines=mandat.findAll('dd')
    lines=re.sub('<[^<]+?>', '', str(lines))
    lines=re.sub('\t', '', str(lines))
    lines=re.sub("\n", '', str(lines))
    line=''
    
    #On met tout dans un dataframe
    columns=columns[1:-1].split(',')
    lines=lines[1:-1].split(',')
    df=pd.DataFrame([lines],columns=columns)
    return df

In [34]:
def RecupNom (page):
    nom_depute=page.find('title')#on va là où il y a le nom
    nom_depute=re.sub('<[^<]+?>', '', str(nom_depute))#on enlève les balises
    nom_depute=nom_depute.split()#on récupère les mots du titre
    mot=nom_depute[0]#on initialise au premier mot
    nom_dep=''#on initialise la variable où on va récupérer le nom
    while mot!='-' :#on prend les premiers mots qui composent le nom, avant la suite du titre
        nom_dep=nom_dep+' '+mot#on concatène le mot appartenant au nom avec les précédents
        nom_depute=nom_depute[1:]#on supprime le mot étudié
        mot=nom_depute[0]#pour passer au suivant
    return nom_dep#on retourne le mot complet. Il est surement possible de le faire beaucoup plus vite en regex, mais les miennes ne marchent pas.

In [ ]:
page=Scrap('http://www2.assemblee-nationale.fr/sycomore/fiche/(num_dept)/'+str(5))
listeMandat=page
liste=[]
numProbleme=[]
for i in range(1,1000) :
    page=Scrap('http://www2.assemblee-nationale.fr/sycomore/fiche/(num_dept)/'+str(i+7))
    #Une petite blague c'est glissé dans le site : les mandats de sénateurs sont présent dans le code html mais ils sont "cachés"
    #On les enlève donc d'abord
    pouf=page.find('div', {'id':"mandats_an"})
    if pouf :
        depute=pouf.findAll('dl', {'class' : "deputes-liste-attributs sycomore"})
    else :#éviter les cas d'assemblée nationale constituante qui ont un autre nom que mandats_an
        depute=page.findAll('dl', {'class' : "deputes-liste-attributs sycomore"})
    for mandat in depute :
        try:
            df = scrap_un_mandat(mandat)
            df["nom du député"]=RecupNom(page)
        except AssertionError :
            numProbleme.append(i)
        else :
            if len(liste)==0: #On initialise avec le premier mandat
                liste=df  
            else:
                liste=pd.concat([liste,df])#on fusionne les mandats
print(len(numProbleme))
            

C:\Users\sylva\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Il y des pages de députés jusqu'à 19500 sur le site.

In [43]:
i

191

In [41]:
liste

,Département,Groupe,Législature,Mandat,Régime politique,nom du député
0,Haute-Garonne,Républicain radical et radical-socialiste,XIVe législature,Du 29 avril 1928 au 31 mai 1932,Troisième République - Chambre des députés,Jean-Baptiste Amat
0,Haute-Garonne,Républicain radical et radical-socialiste,XVe législature,Du 8 mai 1932 au 31 mai 1936,Troisième République - Chambre des députés,Jean-Baptiste Amat
0,Bouches-du-Rhône,Parti socialiste,XIVe législature,Du 26 octobre 1930 au 31 mai 1932,Troisième République - Chambre des députés,Toussaint Ambrosini
0,Bouches-du-Rhône,Parti socialiste,XVe législature,Du 8 mai 1932 au 31 mai 1936,Troisième République - Chambre des députés,Toussaint Ambrosini
0,Calvados,Union pour la démocratie française,IXe législature,Du 9 mars 1991 au 1er avril 1993,Cinquième République - Assemblée nationale,Nicole Ameline
0,Calvados,Union pour la démocratie française et du centre,Xe législature,Du 2 avril 1993 au 18 juin 1995,Cinquième République - Assemblée nationale,Nicole Ameline
0,Calvados,Union pour la démocratie française et du centre,Xe législature,Du 17 décembre 1995 au 21 avril 1997,Cinquième République - Assemblée nationale,Nicole Ameline
0,Calvados,Union pour la démocratie française,XIe législature,Du 1er juin 1997 au 7 juin 2002,Cinquième République - Assemblée nationale,Nicole Ameline
0,Calvados,Union pour la majorité présidentielle,XIIe législature,Du 19 juin 2002 au 18 juillet 2002,Cinquième République - Assemblée nationale,Nicole Ameline
0,Calvados,Union pour un Mouvement Populaire,XIIIe législature,Du 20 juin 2007 au 19 juin 2012,Cinquième République - Assemblée nationale,Nicole Ameline


In [33]:
page=Scrap('http://www2.assemblee-nationale.fr/sycomore/fiche/(num_dept)/'+str(5))
#Une petite blague c'est glissé dans le site : les mandats de sénateurs sont présent dans le code html mais ils sont "cachés"
#On les enlève donc d'abord
pouf=page.find('div', {'id':"mandats_an"})
depute=pouf.findAll('dl', {'class' : "deputes-liste-attributs sycomore"})


François Abadie - Base de données des députés français depuis 1789 - Assemblée nationale


' François Abadie'

In [22]:
page

<!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="fr-FR"> <![endif]--><!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8" lang="fr-FR"> <![endif]--><!--[if IE 8]>         <html class="no-js lt-ie9" lang="fr-FR"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js" lang="fr-FR"> <!--<![endif]--> <head>
<meta charset="utf-8"/>
<title>François Abadie - Base de données des députés français depuis 1789 - Assemblée nationale</title>
<meta content="text/html; charset=utf-8" name="Content-Type"/>
<meta content="fr-FR" name="Content-language"/>
<meta content="IE=edge,chrome=1" name="X-UA-Compatible"/>
<meta content="width=954" name="viewport"/>
<meta content="Assemblée Nationale" name="author"/>
<meta content="Assemblée Nationale" name="copyright"/>
<meta content="Presentation de l'Assemblee nationale, du palais Bourbon, de ses membres (deputes), de son fonctionnement et de son actualite : agenda, travaux en cours (amendements, rapports, commissions, lo

In [35]:
#On fusionne les mandats
liste=[]
for mandat in depute :
    df = scrap_un_mandat(mandat)
    if len(liste)==0: #On initialise avec le premier mandat
        liste=df      
    else:
        liste=pd.concat([liste,df])
liste["nom du député"]=RecupNom(page)
liste

,Régime politique,Législature,Mandat,Département,Groupe,nom du député
0,Cinquième République - Assemblée nationale,Ve législature,Du 2 avril 1973 au 2 avril 1978,Hautes-Pyrénées,Parti socialiste et radicaux de gauche,François Abadie
0,Cinquième République - Assemblée nationale,VIe législature,Du 19 mars 1978 au 22 mai 1981,Hautes-Pyrénées,Socialiste,François Abadie
0,Cinquième République - Assemblée nationale,VIIe législature,Du 2 juillet 1981 au 23 juillet 1981,Hautes-Pyrénées,Socialiste,François Abadie
